In [1]:
import pandas as pd
import csv
import codecs
import os
import geopy

In [2]:
tree = pd.read_csv("tree-planting-locations.csv", delimiter=';')
# tree = tree[['Feature start date', 'Type of tree', 'geo_point_2d']]
tree.head(5)

,OBJECTID,X,Y,Site code,Site name,Plot number,Sponsorship,Latin code,Latin name,Feature type name,Type of tree,Common name,Asset id,Feature start date,geo_shape,geo_point_2d
0,915777,353494.02,177677.48,BANGWAAL,Bangrove Walk Amenity Land,200013.5,Available to Sponsor,LITU,Liriodendron tulipifera,PK: Tree - Tree Planting Locat,Tulip Tree,Tulip tree,PK00285796,2017-08-08T03:00:00+01:00,"{""coordinates"": [-2.671314279945971, 51.496112...","51.496112976681, -2.671314279945971"
1,915772,354057.17,177113.99,SHIRPA,Shirehampton Park,200001.5,Available to Sponsor,ACC2,Acer campestre,PK: Tree - Tree Planting Locat,Field Maple,Field maple,PK304407,2019-09-26T03:00:00+01:00,"{""coordinates"": [-2.663128971955957, 51.491093...","51.49109301287853, -2.663128971955957"
2,915731,355172.20,175614.16,4540939,Sabrina Way,200005.5,Available to Sponsor,SOA4,Sorbus a 'Cardinal Royal',PK: Tree - Tree Planting Locat,Rowan,Rowan cultivar,PK00287664,2017-10-23T03:00:00+01:00,"{""coordinates"": [-2.646878977336136, 51.477698...","51.47769804124667, -2.646878977336136"
3,915729,363973.25,175315.93,HILLREGR,Hillfields Park,200015.5,Available to Sponsor,BEPE,Betula pendula,PK: Tree - Tree Planting Locat,Silver Birch,Silver birch,PK00501026,2020-10-08T03:00:00+01:00,"{""coordinates"": [-2.520124020216623, 51.475647...","51.47564706553951, -2.520124020216623"
4,915708,362242.84,175353.07,4560023,Thingwall Park,200022.5,Available to Sponsor,TIC1,Tilia cordata Greenspire,PK: Tree - Tree Planting Locat,Lime,Small-leaved lime cultivar,PK00501033,2020-08-07T03:00:00+01:00,"{""coordinates"": [-2.545042016053264, 51.475867...","51.475867914271525, -2.545042016053264"


In [3]:
tree_zipcode = pd.read_csv('tree_df-zipcode.csv')
tree_zipcode.head(5)

,Unnamed: 0,Year,Month,Day,Hour,geo_point_2d,Plot number,OBJECTID,Longitude,Latitude,ZipCode
0,0,2017,8,8,3,"51.496112976681, -2.671314279945971",200013.5,915777,51.496113,-2.671314,BS11 0EZ
1,1,2019,9,26,3,"51.49109301287853, -2.663128971955957",200001.5,915772,51.491093,-2.663129,BS11 0UL
2,2,2017,10,23,3,"51.47769804124667, -2.646878977336136",200005.5,915731,51.477698,-2.646879,BS9 1ST
3,3,2020,10,8,3,"51.47564706553951, -2.520124020216623",200015.5,915729,51.475647,-2.520124,BS16 4HS
4,4,2020,8,7,3,"51.475867914271525, -2.545042016053264",200022.5,915708,51.475868,-2.545042,BS16 2AN


In [4]:
tree_zipcode = tree_zipcode[['geo_point_2d', 'ZipCode']]
tree_zipcode.head(5)
len(tree_zipcode)

824

In [5]:
tree['Date'] = tree['Feature start date'].apply(lambda x: x.split('+')[0])
tree['Year'] = pd.to_datetime(tree['Date'],format='%Y-%m-%dT%H:%M:%S').dt.year
tree['Month'] = pd.to_datetime(tree['Date'],format='%Y-%m-%dT%H:%M:%S').dt.month
tree['Day'] = pd.to_datetime(tree['Date'],format='%Y-%m-%dT%H:%M:%S').dt.day
tree['Hour'] = pd.to_datetime(tree['Date'],format='%Y-%m-%dT%H:%M:%S').dt.hour

tree = tree[['Year', 'Month', 'Day', 'Hour', 'geo_point_2d', 'Type of tree']]
tree['Longitude'] = tree['geo_point_2d'].apply(lambda x: x.split(',')[0])
tree['Latitude'] = tree['geo_point_2d'].apply(lambda x: x.split(',')[1])

# tree = tree.dropna(subset=['Type of tree'])
tree.head(5)

,Year,Month,Day,Hour,geo_point_2d,Type of tree,Longitude,Latitude
0,2017,8,8,3,"51.496112976681, -2.671314279945971",Tulip Tree,51.496112976681,-2.671314279945971
1,2019,9,26,3,"51.49109301287853, -2.663128971955957",Field Maple,51.49109301287853,-2.663128971955957
2,2017,10,23,3,"51.47769804124667, -2.646878977336136",Rowan,51.47769804124667,-2.646878977336136
3,2020,10,8,3,"51.47564706553951, -2.520124020216623",Silver Birch,51.47564706553951,-2.520124020216623
4,2020,8,7,3,"51.475867914271525, -2.545042016053264",Lime,51.475867914271525,-2.545042016053264


In [6]:
tree = pd.merge(tree, tree_zipcode, on='geo_point_2d')
tree.head(5)

,Year,Month,Day,Hour,geo_point_2d,Type of tree,Longitude,Latitude,ZipCode
0,2017,8,8,3,"51.496112976681, -2.671314279945971",Tulip Tree,51.496112976681,-2.671314279945971,BS11 0EZ
1,2019,9,26,3,"51.49109301287853, -2.663128971955957",Field Maple,51.49109301287853,-2.663128971955957,BS11 0UL
2,2017,10,23,3,"51.47769804124667, -2.646878977336136",Rowan,51.47769804124667,-2.646878977336136,BS9 1ST
3,2020,10,8,3,"51.47564706553951, -2.520124020216623",Silver Birch,51.47564706553951,-2.520124020216623,BS16 4HS
4,2020,8,7,3,"51.475867914271525, -2.545042016053264",Lime,51.475867914271525,-2.545042016053264,BS16 2AN


In [7]:
# tree['Zip Code']=tree['Year']

# geo_locator = geopy.Nominatim(user_agent='1234')

# for i in range(len(tree)):
#     r = geo_locator.reverse((tree.loc[i,'Longitude'], tree.loc[i,'Latitude']))
#     tree.loc[i,'Zip Code'] = r.raw['address']['postcode']

# tree.head(5)

In [8]:
tree = tree.dropna(subset=['Type of tree'])
tree['zipcode'] = tree['ZipCode'].apply(lambda x: x.split(' ')[0])
tree = tree[['Year', 'Month', 'Day', 'Hour', 'Type of tree', 'zipcode']]


In [9]:
typeoftree = tree['Type of tree'].tolist()
typeoftree = set(typeoftree)
typeoftree

{'Alder',
 'Amelanchier',
 'Apple',
 'Armstrong Canadian Maple',
 'Aspen',
 'Beech',
 'Birch',
 'Box Elder',
 'Bristol Whitebeam',
 'Catalpa',
 'Cedar',
 'Cherry',
 'Chitalpa',
 'Copper Beech',
 'Cornel',
 'Cotoneaster',
 'Cypress',
 'Dawn Redwood',
 'Dogwood',
 'Elm',
 'False Acacia',
 'Field Maple',
 'Foxglove Tree',
 'Gingko',
 'Handkerchief Tree',
 'Hawthorn',
 'Hazel',
 'Hickory',
 'Holly',
 'Honey Locust',
 'Hop Hornbeam',
 'Hornbeam',
 'Horse Chestnut',
 'Ironwood',
 'Judas Tree',
 'Laburnum',
 'Larch',
 'Lime',
 'Magnolia',
 'Maple',
 'Nettle Tree',
 'Oak',
 'Paper Mulberry',
 'Pear',
 'Pine',
 'Plane',
 'Plum',
 'Poplar',
 'Pride of India',
 'Privet',
 'Redwood',
 'Rowan',
 'Scots Pine',
 'Service Tree',
 'Silver Birch',
 'Snowbell',
 'Southern Beech',
 'Strawberry Tree',
 'Sweet Chestnut',
 'Sweetgum',
 'Sycamore',
 'Tulip Tree',
 'Tupelo',
 'Walnut',
 'Whitebeam',
 'Yellowwood',
 'Yew',
 'Zelkova'}

In [10]:
tree['Air clean bility'] = tree['Type of tree']
high_air_purifier = ['Beech', 'Bristol Whitebeam', 'Cedar', 'Cherry', 'Cotoneaster', 'Gingko', 'Hawthorn',
                 'Holly', 'Hornbeam', 'Laburnum', 'Lime', 'Magnolia', 'Maple', 'Nettle Tree', 'Plane',
                 'Privet', 'Rowan', 'Silver Birch', 'Whitebeam', 'Yew', 'Zelkova', 'Amelanchier', 'Birch',
                 'Larch', 'Oak', 'Poplar', 'Tupelo', 'Walnut']
medium_air_purifier = ['Alder', 'Apple', 'Aspen', 'Catalpa', 'Chitalpa', 'Cornel', 'Dogwood', 'Elm', 
                        'False Acacia', 'Foxglove Tree', 'Handkerchief Tree', 'Hazel', 'Hickory', 
                        'Honey Locust', 'Hop Hornbeam', 'Horse Chestnut', 'Ironwood', 'Judas Tree', 
                        'Paper Mulberry', 'Pine', 'Plum', 'Redwood', 'Scots Pine', 'Service Tree',
                        'Snowbell', 'Southern Beech', 'Sycamore、Sweet Chestnut', 'Sweetgum',
                        'Tulip Tree', 'Yellowwood', 'Armstrong Canadian Maple', 'Field Maple', 'Pear', 'Sycamore']
low_air_purifier = ['Box Elder', 'Copper Beech', 'Cypress', 'Dawn Redwood', 'Larch、Poplar', 'Pride of India', 
                    'Strawberry Tree', 'Sweet Chestnut']
new_value = [0, 1, 2]

tree.loc[tree['Air clean bility'].isin(high_air_purifier), 'Air clean bility'] = new_value[0]
tree.loc[tree['Air clean bility'].isin(medium_air_purifier), 'Air clean bility'] = new_value[1]
tree.loc[tree['Air clean bility'].isin(low_air_purifier), 'Air clean bility'] = new_value[2]
tree

,Year,Month,Day,Hour,Type of tree,zipcode,Air clean bility
0,2017,8,8,3,Tulip Tree,BS11,1
1,2019,9,26,3,Field Maple,BS11,1
2,2017,10,23,3,Rowan,BS9,0
3,2020,10,8,3,Silver Birch,BS16,0
4,2020,8,7,3,Lime,BS16,0
...,...,...,...,...,...,...,...
813,2018,6,22,3,Birch,BS9,0
815,2021,2,4,1,Oak,BS16,0
816,2016,7,15,3,Hornbeam,BS11,0
819,2011,10,7,3,Gingko,BS5,0


In [11]:
a = tree['Air clean bility'].tolist()
a = set(a)
a

{0, 1, 2}

In [13]:
# 使用 get_dummies() 方法将每个不同的值转换成一个单独的列
dummies = pd.get_dummies(tree['Air clean bility'])

# 将新的列添加回原始 DataFrame
tree = pd.concat([tree, dummies], axis=1)
print(tree)

tree.to_csv('tree.csv', index=False)

     Year  Month  Day  Hour  Type of tree zipcode Air clean bility  0  1  2  \
0    2017      8    8     3    Tulip Tree    BS11                1  0  1  0   
1    2019      9   26     3   Field Maple    BS11                1  0  1  0   
2    2017     10   23     3         Rowan     BS9                0  1  0  0   
3    2020     10    8     3  Silver Birch    BS16                0  1  0  0   
4    2020      8    7     3          Lime    BS16                0  1  0  0   
..    ...    ...  ...   ...           ...     ...              ... .. .. ..   
813  2018      6   22     3         Birch     BS9                0  1  0  0   
815  2021      2    4     1           Oak    BS16                0  1  0  0   
816  2016      7   15     3      Hornbeam    BS11                0  1  0  0   
819  2011     10    7     3        Gingko     BS5                0  1  0  0   
821  2016      8   16     3     Whitebeam     BS5                0  1  0  0   

     0  1  2  
0    0  1  0  
1    0  1  0  
2    1

/Users/duchenyu/opt/anaconda3/lib/python3.9/site-packages/pandas/core/algorithms.py:798: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  uniques = Index(uniques)
